Wszystko jest obiektem.

Duck Typing - liczy się interfejs a nie typ

W pythonie 2 wyróżna się 2 typy klass - old style i new style. Klasy starego typu są odradzane, jako że są dosyć ograniczone. Różnica pomiędzy definicją klasy nowego a starego typu to dziedziczenie po *object* (klasy nowego typu)

W Pythonie 3 wszystkie klasy są nowego typu - nie ma konieczności dziedziczenia po object, ale można


Definicja klasy:

In [ ]:
from __future__ import print_function
# klasa o nazwie Nazwa dziedzicząca po object i nie posiadająca zadnych atrybutóœ i metod zdefiniowanych przez użytkownika

# Python 2
class Nazwa(object):
    pass

In [ ]:
# Python 3
class Nazwa:
    pass

Wszystkie atrybuty klasy i instancji są publiczne, a określanie atrybutów jako prywatne i protected to tylko konwencja nazewnictwa:

atrybuty zaczynające się od pojedyńczego podkreślnika \_atrybut - są to atrybuty protected
atrybuty zaczynające się od dwóch podkreslników \_\_atrybut - są to atrybuty prywatne - Python wspiera prywatnosć atrybutów przez mechanizm *name mangling* - ale nie ze wzgledu na dostęp z zewnątrz, tylko w celu uniknięcia kolizji nazw w przypadku dziedziczenia

atrybuty zaczynające się i kończące \_\_ (tzw. dunder) to specjalne atrybuty zarezerwowane dla interpretera (np. przeciążanie operatorów itp.)


In [ ]:
class A(object):
    def __init__(self, x):
        self.__x = x
        self.y = 2 * x
        self._z = x ** 2
        
a = A(1)
# name mangling przed nazwą atrybutu doczepia _ z nazwą klasy (żeby uniknąć kolizji w klasach dziedziczących)
dir(a)

Atrybuty obiektu przechowywane są w specjalnym słowniku \_\_dict\_\_. Słownik ten możńa modyfikować:

In [ ]:
print(A.__dict__)

print(a.__dict__)

a.__dict__['a'] = 13
print(a.__dict__)

W bloku klasy można też umieszczać wywołania funkcji i inny kod pythonowy

In [ ]:
class A(object):
    print('blok')
    2 + 2

Klasa po utworzeniu jest przekazywana do dekoratorów (jeżlei zostały sprecyzowane)

In [ ]:
def dekorator(cls):
    print(cls)
    return cls


@dekorator
class A(object):
    pass

## Monkey Patching
  
To dodawanie metod po definicji klasy, w praktyce wyglada to następująco:

In [ ]:
class A(object):
    pass

a = A()

def f(self):  # wyglada jak metoda, ale jest definiowane poza blokiem klasy
    print(self)
    
A.f = f  # przypięcie funkcji do klasy, to utworzenie nowej metody


a.f()  # wywołanie metody - zdefiniowanej poza blokiem klasy

Jak dodajemy metody do instancji to bez parametru self:

In [ ]:
class A(object):
    pass

a = A()

def f(self):  # taka sama definicja jak wcześniej
    print('f')
    
    
a.f = f  # przypinamy do instancji

a.f()  # Błąd - brakuje argumentu *self* - f to nie metoda tylko wywoływalny atrybut!

In [ ]:
# próba obejścia:
# A, f z poprzedniej komórki

from functools import partial

a.f = partial(f, a)  # wrzucamy obiekt *a* jako parametr funkcji f
a.f()  # działa

**UWAGA - Python 3**

W przypadku definiowania metody na zewnątrz klasy - nie można używać *super* bez argumentów (składnia z Pythona 3) 
i potem przypinać tej metody - skutkuje to RuntimeError:

In [ ]:
# Python 3

def metoda(self):
    super()
    

class A:
    pass

A.metoda = metoda
a = A()
a.metoda()  # RuntimeError

In [ ]:
# Python 3
# użycie składni takiej jak w Pythonie 2 działa - atrybut __class__ jest opisany niżej

def metoda(self):
    super(self.__class__, self)
    

class A:
    pass

A.metoda = metoda
a = A()
a.metoda()  # działa

## Magiczne (dunder) Metody

\_\_new\_\_ - tworzenie obiektu - jest to metoda klasy z automatu (na etapie tworzenia instancji, jeszcze przecież jej niema)

\_\_init\_\_ - inicjalizacja

\_\_str\_\_, \_\_repr\_\_, \_\_unicode\_\_, \_\_format\_\_ - reprezentacja tekstowa

\_\_iter\_\_ - rzutowanie na listę/krotkę


### 3 typy operatorów:
* "normalne": \_\_add\_\_, \_\_mul\_\_
* "i" - inplace: \_\_iadd\_\_, \_\_imul\_\_
* "r" - reversed: \_\_radd\_\_, \_\_rmul\_\_


\_\_call\_\_ - wywoływanie obiektu jak funkcji

\_\_getitem\_\_, \_\_setitem\_\_, \_\_delitem\_\_ - dostęp jak do słownika

\_\_getattr\_\_, \_\_setattr\_\_, \_\_delattr\_\_, \_\_getattribute\_\_ - dostęp do atrybutów. Jeżeli przy dostępie do atrybutu, wyrzucony zostanie wyjątek AttributeError, to nasępuje wywołanie metody \_\_getattr\_\_ z nazwą feralnego atrybutu (ale nie przy usuwaniu nieistniejącego atrybutu).

\_\_hash\_\_ - wyznaczanie hashu obiektu - metoda wywoływana przez wbudowaną funkcję hash(). \_\_hash\_\_ i \_\_eq\_\_ są potrzebne do umieszczenia obiektu w zbiorze/słowniku. Jeżeli obiekt nie posiada metody \_\_eq\_\_ to nie powienien też posiadać metody \_\_hash\_\_. 
Jeżeli posiada obie, to \_\_hash\_\_ powinien byc ten sam dla obiektów, które są równe.
Obiekt możę mieć zdefiniowane samo \_\_eq\_\_, ale wtedy nie jest obiektem haszowalnym.
Jeżeli obiekt jest mutowalny, nie powinien mieć zdefiniowanego \_\_hash\_\_. 
Obiekty użytkownika mają domyślnie zdefiniowane obydwie metody w ten sposób, że wszystkie instancje są uznawane za różne a \_\_hash\_\_ oblicznay jest na podstawie miejsca w pamięci - jeżeli obikty są równe to mają też ten sam \_\_hash\_\_ i id. Jeżeli obiekt definiuje tylko \_\_eq\_\_, to \_\_hash\_\_ jest ustawiane na None. Jeżeli klasa potomna implementuje \_\_eq\_\_ ale chce zachować \_\_hash\_\_ rodzica, to trzeba to explicite ustawić: \_\_hash\_\_ = Parent.\_\_hash\_\_. Jeżeli klasa chce być siłą niehaszowalna, należy przypisać None: \_\_hash\_\_ = None. Domyślnie hash bytes'ów, stringów i datetime jest "solony" - w celu uniknięcia powtarzalności haszu pomiędzy kolejnymi uruchomieniami interpretera i możliwości takiego spreparowania stringów, że czas wsadzania do słownika ma złożoność O(n^2): http://ocert.org/advisories/ocert-2011-003.html




### sekwencje
definiuje:

\_\_len\_\_ - długosć sekwencji

\_\_getitem\_\_ - wybieranie elementu z sekwencji - liczba lub slice

jeżeli sekwencja nie ma metody \_\_iter\_\_ to iterowanie następuje przez wrzucanie kolejnych liczb całkowitych od 0 do metody \_\_getitem\_\_ aż do skutku.

sekwencja może dodatkowo zdefiniować metodę \_\_contains\_\_ do obsługi operatora **in**


### obiekt iterowalny
definiuje:

\_\_iter\_\_ - ma zwracać iterator a nie sekwencję (np. listę)

\_\_getitem\_\_  - jeżeli w klasie nie ma \_\_iter\_\_ to przy iteracji pętlą for python wrzuca liczby od 0, aż nie zostanie wyrzucony wyjątek StopIteration


### iterator:
definiuje:

\_\_next\_\_ (Python3), next (Python2)- zwraca kolejny element obiektu po którym ma iterować
\_\_iter\_\_ - zwraca self


### Typ logiczny:
\_\_bool\_\_ - wywoływane przez funkcję bool(); służy do sprawdzania prawdziwości logicznej obiektu; jeżeli obiekt nie ma \_\_bool\_\_ sprawdzane jest \_\_len\_\_. Jeżeli nie ma żadnej z tych metod, to obiekt ma wartosć True. \_\_bool\_\_ powinien zwracać True albo False


### inne atrybuty i metody

\_\_class\_\_  - przechowuje klasę obiektu. np. a = A(); to a.\_\_class\_\_ to A (obiekt, a nie string)


## NotImplemented vs NotImplementedError
NotImplemented jest obiektem, który powinna zwracać metoda jezeli nie implementuje danej operacji (np dodawanie), ponieważ wtedy interpreter jest wstanie podjąć akcje mające na celu zwrócenie wyniku (np w przypadku dodawania).

NotImplementedError to bład, który jest wyrzucany, kiedy interpreter nie jest juz w stanie zapewnić wyniku operacji - można go obsłuzyć:

In [ ]:
class A(object):    
    def __eq__(self, other):
        print('A.__eq__')
        if not isinstance(other, A):
            return NotImplemented
        return True
        

class B(object):
    def __eq__(self, other):
        print('B.__eq__')
        if not isinstance(other, B):
            return False
        return True
    
a = A()
b = B()

a == b  # klasa A zwraca NotImplemented - to próba porównania używając metody z klasy B

## Atrybuty Klasy

Tak jak instancja ma atrybuty, tak i klasa może mieć atrybuty.

Atrybuty klasą są współdzielone pomiędzy instancjami (coś w stylu atrybutów statycznych).

Atrybuty klasy definiowane są w bloku klasy/przyczepiane do klasy:

In [ ]:
class A(object):
    x = 2
    y = 3
    
A.z = 4  # metody też tak były przyczepiane

print(A.x, A.y, A.z)

Do atrybutów klasy można odwoływać się przez klasę i przez instancję (jezeli instancja nie ma juz zdefinowanych atrybutów o tej samej nazwie)

In [ ]:
a = A()

print(A.x)
print(a.x)

a.x = 4  # przysłonięcie atrybutu klasy - przypisanie obiektu do atrybutu instancji o tej samej nazwie co
         # atrybut w klasie przysłąnia atrybut klasy
print(a.x)

In [ ]:
b = A()
print(b.x)

print(a.y)
print(b.y)

A.y = 33
print(a.y)
print(b.y)

print('A.z, a.z, b.z', A.z, a.z, b.z)  # wszystkie obiekty mają ten sam atrybut z - wzięty z klasy

W przypadku kiedy w instancji jest przysłonięty atrybut klasy, można atrybut klasy wybrać bezpośrednio z klasy używając \_\_class\_\_:

In [ ]:
class A(object):
    x = 33
    
    def f(self):
        return self.x
    
    def g(self):
        return self.__class__.x
    
a = A()
a.x = 44

print(a.x)
print(a.__class__.x)

### Jak odróżnić atrybuty klasy od atrybutów instancji?


Atrybuty instancji są przypisywane do instancji - są to wszystkie atrybuty przypisane wewnątrz metod do *self*, np ```self.x = 33``` - ten kod tworzy atrybut instancji o nazwie *x* i przypisuje mu wartosć 33,
oraz atrybuty przypisane do instancji poza metodami, np: ```a = A(); a.x = 33``` - stworzenie nowego atrybutu instancji *a* o nazwie *x* i przypisanie mu wartości 33



Z kolei Atrybuty klasy, to atrybuty stworzone w bloku klasy, np:
```
class A(object):
    x = 33
```
tutaj to utworzenie klasy z atrybutem klasy o nazwie *x* i wartości 33.
Atrybuty klasy można tworzyc również przez bezpośrednie dopisywanie atrybutów do klasy:

A.x = 33  - stworzenie atrybutu klasy o nazwie *x* i przyisaniemu wartości 33.
ponieważ do klasy można sie również dobrać przez atrybut \_\_class\_\_ to można atrybut klasy ustawić też następujaco:
```
a = A()
a.__class__.x = 33
```
tak jak wcześniej - ustawiamy atrybut klasy o nazwie *x* na wartosć 33

### classmethods i staticmethods

Poza standardowymi metodami, istnieja też metody statyczne i klasy.

classmethods przyjmują jako pierwszy argument klasę (a nie instancję) a metody statyczne nie przyjmują żadnej z powyższych. Metody statyczne i klasowe tworzy się używając dekoratorów. Obydwa typy metod można wywoływać zarówno z poziomu klasy i instancji - mają ten sam efekt

In [ ]:
class A(object):
    @classmethod
    def f(cls):  # cls zamiast self
        print(cls)
    
    @staticmethod
    def g():  # brak self i cls
        print('g')
        

a = A()
a.f()  # wypisze nazwę klasy i tak
A.f()
a.g()  # wypisze g
A.g()

## Dziedziczenie

W Pythonie klasy bazowe są określane w nawiasie koło nazwy klasy. W Pythonie jedna klasa może mieć wiele klas bazowych.
W celu określenia kolejności poszukiwania metod używa się mro (method resolution order)

In [ ]:
class A(object):
    def f(self):
        print("A.f")
    pass

class B(object):
    pass

class C(A):
    def f(self):
        print("C.f")
        pass
        
class D(B):
        pass
        
class E(C, D):
    pass
        
        
print(E.mro())  # metoda mro() pokazuje kolejkę klas do przeszukiwania - w przypadku braku atrybutu/metody w
                # jednej z nich Python przeszukuje następną w kolejności - jeżeli nie ma takiego atrybutu/metody
                # wyrzucany jest AttributeError
E().f()

In [ ]:
# metoda super wywołuje inną metodę z kalsy bazowej - w Pythonie 2 obowiązkowe jest podanie 2 argumentów, 
# w pythonie 3 można wywoływać bez argumentów (ale z argumentami też można)

class A(object):
    def f(self):
        print('A.f')
        super(A, self).f()  # będzie AttributeError bo chce wywołać metodę f z obiektu object - a tam nie ma takiej

class B(object):
    def f(self):
        print('B.f')
        super(B, self).f()

class C(A):
    def f(self):
        print('C.f')
        super(C, self).f()
        
class D(B):
    def f(self):
        print('D.f')
        super(D, self).f()
        
class E(C, D):
    def f(self):
        print('E.f')
        super(E, self).f()
        
class F(C, D):
    def f(self):
        print('F.f')
        super(C, self).f()
        
        
print(E.mro())
try:
    E().f()
except AttributeError as e:
    print('AttributeError: ', e)

print()
print()
    
print(F.mro())
try:
    F().f()
except AttributeError as e:
    print('AttributeError: ', e)

## Managery kontekstu

Managery kontekstu funkcjonują w połaczeniu z wyrażeniem *with*.
Interfejs managera to 2 metody: \_\_enter\_\_ i \_\_exit\_\_.

Obiekt, który będzie zwrócony w metodzie enter będzie przypisany do zmiennej po 'as'.

Metoda \_\_exit\_\_ przyjmuje informacje o wyjątku jaki został wyrzucony w bloku kontekstu i nie obsłuzony przez nic (albo same None jeżeli nie byo wyjątku) a zwraca True - jeżeli (ewentualny) wyjątek został przez nią obsłużony albo False jeżeli nie został - taki wyjątek bedzie ponownie wyrzucony

In [ ]:
class Manager(object):
    def __enter__(self):  # __enter__ podczas wejscia do kontekstu - with
        print('Manager.__enter__')
        return 22  # to co tu zostanie zwrócone będzie przypisane do zmiennej
    
    def __exit__(self, exc_type, exc_value, traceback):
        # exc_type to klasa wyjątku
        # exc_value to instancja wyjątku
        # traceback to traceback
        if exc_type is KeyboardInterrupt:
            print('ctrl c')
            return True  # oznaczamy wywołanie jako będące sukcesem
        print('Exception not handled')
        return False
        
        
with Manager() as x:
    print(x)  # nic nie wyrzuci wyjątku w bloku - metoda __exit__ zwróci False, ale ponieważ nie było wyjątku to
              # nic się nie stanie


In [ ]:
with Manager() as x:
    print(x)
    raise KeyboardInterrupt('stop')  # KeyboardInterrupt jest obsłużony w metodzie __exit__ - dla neigo metoda
                                     # zwraca True

In [ ]:
with Manager() as x:  # nie obsługuję błędów automatycznie
    print(x)
    {}[1]  # KeyError nie jest obsłużony - dla niego metoda __exit__ zwróci False

In [ ]:
# managery kontekstu mozna inicjalizować poza blokiem with:

man = Manager()
with man as x:  # to samo co with Manager() as x
    print(x)

In [ ]:
# można używać kilku managerów w jendym kontekscie:

class Manager(object):
    def __init__(self, id_):
        self.id_ = id_
        
    def __enter__(self):
        print('Manager id:', self.id_)  # na potrzeby identyfikacji managerów
        return self.id_
    
    def __exit__(self, *exc_info):
        return True
    

with Manager(1) as m1, Manager(2) as m2, Manager(3), Manager(4) as m4:  # nie jest wymagane przypisywanie wyników
                                                                        # __enter__ do zmiennych
    print(m1)  # 1
    print(m2)  # 2
    print(m4)  # 4
    
# Manager 3 się tylko wypisał - wynik __enter__ w tym przypadku nie był przypisany do żadnej zmiennej

In [ ]:
# alternatywnie można użyć dekoratora - contextmanager
# to co jest przed yield będzie wykonywane jako __enter__
# to co jest yield'owane będzie przypisane do zmiennej po 'as'
# to co po yield bedzie traktowane jak __exit__
from contextlib import contextmanager

@contextmanager
def manager(arg):
    print('__enter__')
    print(arg)
    yield arg ** 2
    print('__exit__')
    
with manager(2) as x:
    print(x)


# najcześćiej taki manager kontekstu przyjmuje następującą formę (try/except pozwala obsługiwać wyjątki):
@contextmanager
def manager(arg):
    print('__enter__')
    print(arg)
    try:
        yield arg ** 2
    except KeyError:
        print('obsługa KeyError')
    print('__exit__')
    
    
with manager(2) as x:
    {}[1]  # KeyError, ale będzie obsłużony

## Deskryptory
Deskryptory to klasy, które maja zaimplementowaną co najmniej jedną z metod: \_\_get\_\_, \_\_set\_\_.
Mogą mieć również zaimplementowane \_\_delete\_\_.

Desktyptory służą do obsłużenia przypisywania, odczytywania i usuwania atrybutu.
Pozwalają zachować interfejs klasy bez zmian w wypadku, kiedy do wyznaczenia jakiegoś atrybutu jest potrzeba wykonania jakiejś operacji, a wcześniej nie trzeba było (i był to zwykły obiekt - np int/str)
Przypisywane są do atrybutu klasy:

### Ogolny flow deskryptorów

normalnie, kiedy atrybut ```a.x``` nie jest deskryptorem, to wyszukiwanie atrybutu odbywa się w kolejności:
1. przeszukiwanie instancji
2. przeszukiwanie klasy
3. przeszukiwanie klas bazowych
Metaklasy nie są przeszukiwane.

Jeżeli atrybut jest deskryptorem to flow jest inny niż normalnie:
* można wywołać desktyptor ręcznie ```x.__get__(a)```
* przy wywołaniu ```a.x``` jest transformowane do ```type(a).__dict__['x'].__get__(a, type(A))```
* ```A.x``` jest przekształcane do ```A.__dict__['x'].__get__(None, A)```
* przy wywoływaniu desktyptora z klasy bazowej (przez super lub nie) do deksryptora przekazywane są instancja i klasa z której jest to wywoływane a nie klasa bazowa

In [ ]:
class Deskryptor(object):
    def __get__(self, instance, cls):
        # instance to insatncja klasy, do której jest przypisany deskryptor i na rzecz której odbywa się odczyt
                # atrybutu-deskryptora
        # cls to klasa do której jest przypisany deskryptor
        print('get:')
        print(self, instance, cls)
        
    def __set__(self, instance, value):
        # instance - tak jak prz __get__
        # value - obiekt, który jest przypisywany do atrybutu-deskryptora
        print('set:')
        print(self, instance, value)
        
    def __delete__(self, instance):
        # instance - j.w.
        print('delete:')
        print(self, instance)
        
        

class A(object):
    attr = Deskryptor()  # instancja deskryptora przypisana do atrybutu klasy
    

a = A()
a.attr  # odczytywanie atrybutu klasy (będącego deskryptorem) z instancji
a.attr = 24  # próba przysłonienia atrybutu klasy (deskryptora) w instancji
a.attr  # nie wyszło - dalej jest to deskryptor
del a.attr

# a.attr - tutaj do metody __get__ deskryptora jest przekazywane - *instance* to obiekt *a*, *cls* to klasa *A*
# a.attr = 24 - tutaj do metody __set__ jako *instance* jest przkazywany obiekt *a* a jako value liczba 24
# del a.attr - do metody __delete__ przekazywany jest obiekt *a* jako instance
# w każdym z powyższych przypadków, self to instancja deskryptora czyli attr

W Pythonie 3.6 w deksryptorze można zaimplementować metodę \_\_set\_name\_\_, do której przekazywana jest klasa i nazwa atrybutu klasy, który będzie deskryptorem

In [ ]:
# Python 3.6

class Deskryptor:
    def __set_name__(self, class_, name):
        print('class:', class_, 'name:', name)

class A:
    d = Deskryptor()

Do pisania deskryptorów nie jest konieczne pisanie klasy od poczatku - można zastosować dekorator *property* i stworzyć deskryptor z funkcji

In [ ]:
class A(object):
    @property
    def x(self):  # nowy deskryptor o nazwie x - jest to przy okazji definicja metody __get__
        print('x getter')
        return 23
    
    @x.setter  # definicja metody __set__ - uwaga! - w dekoratorze jest nazwa desktyptora!
    def x(self, value):  # taka sama nazwa jak nazwa deskryptora
        print('x setter')
    
    @x.deleter
    def x(self):
        print('x delete')
        
a = A()
a.x
a.x = 24
del a.x



In [ ]:
# alternatywnie:
def getter(self):
    print('x getter')
    return 23
    
def setter(self, value):
    print('x setter')
    
def deleter(self):
    print('x delete')

class A(object):
    x = property(getter, setter, deleter)
    # x = property(fget=getter, fset=setter, fdel=deleter)
        
a = A()
a.x
a.x = 24
del a.x

Kombinacje metod deskryptora - jeżeli deskryptora nie ma metody \_\_get\_\_ to odczyt zwraca niego samego, chyba, że instancja, z której jest odczytywany ma atrybut o takiej nazwie

deskryptor mający \_\_set\_\_ i/lub \_\_delete\_\_ to tzw. data descriptor.

jeżeli deksryptor nie ma powyższych metod to jest to tzw. non-data descriptor - taki deskryptor ma tylko \_\_get\_\_

deskryptor mający \_\_get\_\_ i \_\_set\_\_ daje pełną kontrolę nad dostępem do atrybutu i przysłaniają atrybut o takiej samej nazwie w instancji, natomiast non-data descriptors mogą być przysłaniane w instancjach.

Żeby przyspieszyć odczyt atrybutu, którego obliczenie może być czasochłonne a wartosć nie będzie się zmienać w czasie (np połaczenie sieciowe), można do atrybutu przypisać Deskryptor, który implementuje tylko metodę \_\_set\_\_, oblicza wartosć i przypisuje do atrybutu instancji o tej samej nazwie, lub zaimplementować deskryptor z \_\_get\_\_ i przypisać wartość do atrybutu w instancji. W obydwóch przypadkach, przypisanie atrybutu do instancji odbywa się przez jej \_\_dict\_\_.

Sztuczki te nie działaja w przypadku użycia @property (bo to data-descriptor):

In [ ]:
from time import sleep


class SetDescriptor(object):
    def __init__(self, name):
        self.name = name
        
    def __set__(self, instance, value):
        print('setting')
        sleep(3)
        instance.__dict__[self.name] = value
     
    
class GetDescriptor(object):
    def __init__(self, name):
        self.name = name
        
    def __get__(self, instance, owner):
        print("get")
        sleep(3)
        val = 22
        instance.__dict__[self.name] = val
        return val
    
    
def getter(self):
    print("get")
    sleep(3)
    # self.g_prop = 23  - nie można przypisać wartosci do @property niemającego __set__
    return 23


def setter(self, value):
    print('setting')
    sleep(3)
    self.__dict__['s_prop'] = value
    

class A(object):
    s_desc = SetDescriptor('s_desc')
    g_desc = GetDescriptor('g_desc')
    g_prop = property(fget=getter)
    s_prop = property(fset=setter)
    

a = A()
print(vars(a))
print(a.g_desc)  # oczekiwanie przez 3 sekundy
#a.g_desc = 25
print(a.g_desc)  # wartosć otrzymana od razu
print(vars(a))



In [ ]:
a = A()
print(vars(a))  # instancja nie ma żadnych dodatkowych atrybutów
print(a.s_desc)  # <__main__.SetDescriptor object at ...>
a.s_desc = 55
print(a.s_desc)  # teraz odczyt atrybutu instancji, ponieważ deskryptor nie ma metody __get__
print(vars(a))  # instancja ma dodatkowy atrybut


In [ ]:
a = A()
print(vars(a))
print(a.g_prop)
#a.g_prop = 25  # AttributeError Can't set attribute
print(a.g_prop)
print(vars(a))

In [ ]:
a = A()
print( vars(a))
#print(a.s_prop)  # AttributeError - Unreadable attribute
a.s_prop = 25
print(vars(a))  # s_prop jest w słowniku, ale nie można tego odczytać metodą "konwencjonalną"
print(a.s_prop)  # AttributeError - Unreadable attribute - pomimo przypisania do atrybutu - @property blokuje dostęp

# Python najpierw sprawdza czy dany atrybut nie jest deskryptorem i jezeli nie jest to dopiero 
# sprawdza atrybut w instancji -> klasie -> klasach bazowych

### Przykład klasy z zaimplementowanymi metodami specjalnymi

In [ ]:
class AIterator(object):
    """
    iterator do iteracji po obiekcie A
    """
    def __init__(self, a):
        self.a = a
        self.fields = ['x', 'y', 'z']
        self.index = 0
    
    def __iter__(self):
        """
        metoda __iter__ iteratora powinna zwracać self
        """
        return self
    
    def __next__(self):
        if self.index < len(self.fields):
            self.index += 1
            return self.fields[self.index - 1]
        else:
            raise StopIteration()
            
    next = __next__  # kompatybilne z python2 i 3


class A(object):
    """
    obiekt A
    """
    def __new__(cls, x, y, z):  # __new__ to metoda klasy, chociaż nie jest bezpośrednio udekorowana
        """
        __new__ to metoda klasy, chociaż nie jest dekorowana @classmethod
        """
        return super(A, cls).__new__(cls)
    
    def __init__(self, x, y, z):
        """
        __init__ inicjalizuje instancję
        """
        self.x = x
        self.y = y
        self.z = z
        
    def __unicode__(self):
        """
        __unicode__ rzutuje na unicode
        """
        return u'unicoded'
    
    def __str__(self):
        """
        __str__ rzutuje na str
        """
        return 'stringed'
    
    def __format__(self, spec):
        """
        '{:error}'.format(self) -> self.__format__('error')
        """
        if spec == 'error':
            raise Exception()
        return 'A(x={}, y={}, z={})'.format(self.x, self.y, self.z)
    
    def __float__(self):
        """
        rzutuje na float
        """
        return self.x
    
    def __int__(self):
        """
        rzutuje na int
        """
        return int(float(self))
    
    def __hash__(self):
        """
        haszuje na potrzeby zbioru i słownika
        """
        return int(self) ** 2
    
    def __eq__(self, other):
        """
        self == other
        """
        return self.x == other.x
    
    def __add__(self, other):
        if isinstance(other, A):
            return self.x + other.x
        return NotImplemented
    
    __radd__ = __add__ # alias - teraz a.__radd__ to samo co a.__add__
    
    def __iter__(self):
        """
        zwraca iterator do iterowania po obiekcie i rzutowania na listę/krotkę.
        """
        return (i for i in [self.x, self.y, self.z])

#    alternatywna implementacja
#    def __iter__(self):
#        """
#        zwraca iterator do iterowania po obiekcie.
#        """
#        for i in [self.x, self.y, self.z]:
#            yield i

#    alternatywna implementacja 2
#    def __iter__(self):
#        """
#        zwraca iterator do iterowania po obiekcie.
#        """
#        return AIterator(self)

    def __len__(self):
        """
        len(self)
        """
        return 3
    
    def __getitem__(self, item):
        """
        __getitem__ -> self[item]
        jeżeli wywołane self[start:stop:step] to item to slice(start, stop, step)
        """
        
        if isinstance(item, slice):
            return [self.x, self.y, self.z][item]
        else:
            return [self.x, self.y, self.z][item]
        
    def __setitem__(self, item, val):
        """
        __setitem__ -> self[item] = val
        """
        [self.x, self.y, self.z][item] = val
        
    def __getattribute__(self, attr):
        """
        -> self.attr
        """
        return getattr(self, attr)
    
    def __setattr__(self, attr, val):
        """
        -> self.attr = val
        """
        return setattr(self, attr)
    
    def __getattr__(self, attr):
        """
        obsługa braku atrybutu attr, 
        np
        a = A(1, 2, 3)
        a.a -> a.__getattr__('a')
        """
        return 22
    
    def __contains__(self, item):
        """
        item in self
        """
        return item in [self.x, self.y, self.z]
    
    def __dir__(self):
        """
        wywoływana przez funkcję dir. zwrócona sekwencja jest zamieniana na listę i sortowana
        """